# Inteligencia Artificial
# - Laboratorio 8 -

Integrantes
- Mark Albrand 
- Jimena Hernández
- Melissa Perez

## Tasks 1 - Teoría


### 1. Investigar el algoritmo AC-3 y su relación con el algoritmo de backtracking search
### 2. Defina en sus propias palabras el término “Arc Consistency”

## Tasks 2 - CSP con Backtracking, Beam y Local Search

Implementar tres algoritmos diferentes de satisfacción de restricciones para resolver un problema de programación de exámenes para cuatro estudiantes que toman siete exámenes diferentes. El
problema implica calendarizar los exámenes para los estudiantes respetando diversas limitaciones y preferencias.

Restricciones:
- Todos los exámenes deberán realizarse en días diferentes, concretamente lunes, martes y miércoles.
- Ningún estudiante deberá tener más de un examen por día.
- Los estudiantes que toman el mismo curso no pueden tener exámenes programados para el mismo día

### Definición de variables

In [29]:
exams = {"Exam 1", "Exam 2", "Exam 3", "Exam 4", "Exam 5", "Exam 6", "Exam 7"}
students = {"Student 1", "Student 2", "Student 3", "Student 4"}
days = {"Monday", "Tuesday", "Wednesday"}

schedule = {d: {} for d in days}
print(schedule)

{'Monday': {}, 'Tuesday': {}, 'Wednesday': {}}


### Backtracking Search

In [35]:

def backtracking(schedule, exams, students, days):
    def is_valid(schedule, exam, student, day):
        if student in schedule[day].values():
            return False
        if day in schedule and exam in schedule[day]:
            return False
        return True

    if not exams:
        return True
    exam = exams.pop()
    for day in days:
        for student in students:
            if is_valid(schedule, exam, student, day):
                schedule[day][exam] = student
                if backtracking(schedule, exams, students, days):
                    return True
                del schedule[day][exam]
    exams.add(exam)
    return False

schedule_backtracking = {d: {} for d in days}
backtracking(schedule, exams, students, days)

for day, exams in schedule.items():
    print(day)
    for exam, student in exams.items():
        print(f"\t{exam}: {student}")

Monday
	Exam 5: Student 2
	Exam 1: Student 3
	Exam 4: Student 4
	Exam 3: Student 1
Tuesday
	Exam 6: Student 2
	Exam 7: Student 3
	Exam 2: Student 4
Wednesday


### Beam Search

In [37]:
def beam_search(schedule, exams, students, days, beam_width):
    def is_valid(schedule, exam, student, day):
        if student in schedule[day].values():
            return False
        if day in schedule and exam in schedule[day]:
            return False
        return True

    def heuristic(schedule):
        """
        Define la heurística para evaluar la calidad de una solución.
        El valor retornado representa la cantidad de conflictos en el horario.
        """
        return sum(len(set(schedule[day].values())) - len(schedule[day]) for day in days)

    if not exams:
        return True

    exam = exams.pop()
    candidates = []
    for day in days:
        for student in students:
            if is_valid(schedule, exam, student, day):
                schedule[day][exam] = student
                candidates.append((day, student))
                del schedule[day][exam]

    candidates.sort(key=lambda x: heuristic(schedule))
    candidates = candidates[:beam_width]

    for day, student in candidates:
        schedule[day][exam] = student
        if beam_search(schedule, exams, students, days, beam_width):
            return True
        del schedule[day][exam]

    exams.add(exam)
    return False

beam_width = 2

success = beam_search(schedule, exams, students, days, beam_width)

if success:
    for day, exams in schedule.items():
        print(day)
        for exam, student in exams.items():
            print(f"\t{exam}: {student}")
else:
    print("No se encontró solución.")

Monday
	Exam 5: Student 2
	Exam 1: Student 3
	Exam 4: Student 4
	Exam 3: Student 1
Tuesday
	Exam 6: Student 2
	Exam 7: Student 3
	Exam 2: Student 4
Wednesday


### Local search